In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
# import keras
from tensorflow.keras.layers import Dropout,BatchNormalization,LSTM,Bidirectional,GlobalMaxPool1D,Input,Activation,Flatten,Embedding,Dense,concatenate,Conv1D,MaxPooling1D
import string
import re
from tqdm import tqdm
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
import spacy
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,f1_score,recall_score,precision_score,classification_report
import os
from tensorflow.keras.models import Model
import seaborn as sns
from matplotlib import pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import warnings
warnings.filterwarnings('ignore')
import tensorboard
from textblob import TextBlob
import os
import tensorboard
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
import datetime
from tensorflow.keras.initializers import he_normal
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from transformers import TFBertModel, BertTokenizer
import nltk

In [ ]:
nltk.download('omw-1.4')

In [ ]:
data=pd.read_csv("../input/sarcasm/train-balanced-sarcasm.csv")
data.shape

In [ ]:
null_values=data.isna().sum()
print(null_values)

In [ ]:
data=data.dropna()
data.isna().sum()

In [ ]:
duplicate_values=data[data.duplicated()]
print("duplicate rows in the dataset",len(duplicate_values))
data.drop_duplicates(keep='first',inplace=True)

In [ ]:
data['score']=data['ups']-data['downs']

In [ ]:
data.head(5)

In [ ]:
null_values=data.isna().sum()
print(null_values)

In [ ]:
data=data.dropna()
data.isna().sum()

In [ ]:
duplicate_values=data[data.duplicated()]
print("duplicate rows in the dataset",len(duplicate_values))
data.drop_duplicates(keep='first',inplace=True)

In [ ]:
# slangs
strings='''AFAIK=As Far As I Know
AFK=Away From Keyboard
ASAP=As Soon As Possible
ATK=At The Keyboard
ATM=At The Moment
A3=Anytime, Anywhere, Anyplace
BAK=Back At Keyboard
BBL=Be Back Later
BBS=Be Back Soon
BFN=Bye For Now
B4N=Bye For Now
BRB=Be Right Back
BRT=Be Right There
BTW=By The Way
B4=Before
B4N=Bye For Now
CU=See You
CUL8R=See You Later
CYA=See You
FAQ=Frequently Asked Questions
FC=Fingers Crossed
FWIW=For What It's Worth
FYI=For Your Information
GAL=Get A Life
GG=Good Game
GN=Good Night
GMTA=Great Minds Think Alike
GR8=Great!
G9=Genius
IC=I See
ICQ=I Seek you (also a chat program)
ILU=ILU: I Love You
IMHO=In My Honest/Humble Opinion
IMO=In My Opinion
IOW=In Other Words
IRL=In Real Life
KISS=Keep It Simple, Stupid
LDR=Long Distance Relationship
LMAO=Laugh My A.. Off
LOL=Laughing Out Loud
LTNS=Long Time No See
L8R=Later
MTE=My Thoughts Exactly
M8=Mate
NRN=No Reply Necessary
OIC=Oh I See
PITA=Pain In The A..
PRT=Party
PRW=Parents Are Watching
QPSA?=Que Pasa?
ROFL=Rolling On The Floor Laughing
ROFLOL=Rolling On The Floor Laughing Out Loud
ROTFLMAO=Rolling On The Floor Laughing My A.. Off
SK8=Skate
STATS=Your sex and age
ASL=Age, Sex, Location
THX=Thank You
TTFN=Ta-Ta For Now!
TTYL=Talk To You Later
U=You
U2=You Too
U4E=Yours For Ever
WB=Welcome Back
WTF=What The F...
WTG=Way To Go!
WUF=Where Are You From?
W8=Wait...
7K=Sick:-D Laugher'''

In [ ]:
x1=strings.split("\n")
dict1={}
for i in x1:
    x2=(i.split("="))
    dict1[x2[0]]=x2[1]
print(dict1)

In [ ]:
def chat(text):
    new_text=[]
    for word in text.split():
        if word.upper() in dict1:
            new_text.append(dict1[word.upper()])
        else:
            new_text.append(word)
            
    done=" ".join(new_text)

            
    return done
            

In [ ]:
data['comment']=data['comment'].apply(chat)

In [ ]:
#decontract words
def decontracted(phrase):
    
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    
    return phrase

In [ ]:
data['comment']=data['comment'].apply(decontracted)

In [ ]:
# lower case
data['comment']=data['comment'].str.lower()

In [ ]:
# removing stopwords

# we have removed stopwords like no,not,nor.
stop_words=["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", 
            "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", 
            "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", 
            "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being",
            "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but",
            "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", 
            "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", 
            "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", 
            "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few",
            "more", "most", "other", "some", "such" "only", "own", "same", "so", "than", "too", "very", 
            "s", "t", "can", "will", "just", "don", "should", "now"]

In [ ]:
def stopwords1(text):
    new_list=[]
    for word in text.split():
        if word in stop_words:
            new_list.append("")
        else:
            new_list.append(word)

    done=list(filter(None,new_list))
    done=" ".join(done)
    
    return done

In [ ]:
data['comment']=data['comment'].apply(stopwords1)

In [ ]:
# remove html tags
def remove_html(text):
    return re.sub(r'<.*?>',"",text)

In [ ]:
data['comment']=data['comment'].apply(remove_html)

In [ ]:
# removing digits
def remove_numbers(text):
    return re.sub("\d+", "", text)

In [ ]:
data['comment']=data['comment'].apply(remove_numbers)

In [ ]:
string1=string.punctuation
string1=list(string1)
string1.remove('!')
string1.remove('?')
print(string1)

In [ ]:
def remove_punctuation(data):
    for char in string1:
        if char in data:
            data=data.replace(char," ")
    return data

In [ ]:
data['comment']=data['comment'].apply(remove_punctuation)

In [ ]:
data_basic=data.drop(['author','score','ups','downs','date','created_utc','parent_comment','subreddit'],axis=1)

In [ ]:
y=data_basic['label']
X=data_basic['comment']

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,stratify=y,random_state=42,test_size=0.3,shuffle=True)
print((X_train.shape),y_train.shape)
print((X_test.shape),y_test.shape)

In [ ]:
bert_model=TFBertModel.from_pretrained('bert-base-uncased')
bert_tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
def text_encoding(tokenizer, texts, max_length, batch_size=512):
    
    input_ids = []
    attention_mask = []
    
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        inputs = tokenizer.batch_encode_plus(batch,
                                             max_length=max_len, padding='max_length',
                                             truncation=True, return_attention_mask=True,
                                             return_token_type_ids=False)
        input_ids.extend(inputs['input_ids'])
        attention_mask.extend(inputs['attention_mask'])        
    
    
    return tf.convert_to_tensor(input_ids), tf.convert_to_tensor(attention_mask)

In [ ]:
max_len = 100
X_train_ids, X_train_attention = text_encoding(bert_tokenizer, X_train.tolist(),max_len)
X_test_ids, X_test_attention = text_encoding(bert_tokenizer, X_test.tolist(), max_len)

In [ ]:
for layer in bert_model.layers:
    layer.trainable=False

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
def model_architecture():
    # Model architecture

    weight_initializer = tf.keras.initializers.GlorotNormal(seed=42)

    # DistilBERT Layers
    input_ids_layer = Input(shape=(max_len,), name='input_ids', dtype='int32')
    input_attention_layer = Input(shape=(max_len,), name='input_attention', dtype='int32')
    last_hidden_state = bert_model(input_ids_layer,input_attention_layer)[0]
#     cls_token = last_hidden_state[:, 0, :]

    # NN layer
    lstm_layer = LSTM(100,name='LSTM')(last_hidden_state)
    X = Dropout(0.4)(lstm_layer )
    X = Dense(128, activation='relu')(X)
    X = Dropout(0.4)(X)
    X = Dense(64, activation='relu')(X)
    X = Dropout(0.4)(X)
    X = Dense(16, activation='relu')(X)
    
    output = Dense(1,activation='sigmoid',kernel_initializer=weight_initializer)(X)

    model_bert = Model(inputs=[input_ids_layer, input_attention_layer], outputs=output)
    
    return model_bert

In [ ]:
model_bert=model_architecture()
# model_bert.layers[2].trainable=True

In [ ]:
model_bert.summary()

In [ ]:
model_bert.compile(loss='binary_crossentropy',optimizer=Adam(lr=0.0001),metrics=['accuracy'])

In [ ]:
Epochs=10
Batch_size=512

model_bert_results = model_bert.fit(
    x = [X_train_ids, X_train_attention],
    y = y_train,
    epochs = Epochs,
    batch_size = Batch_size,
    validation_data = ([X_test_ids, X_test_attention], y_test), callbacks=[callback])

In [ ]:
# Load the library
from tensorflow.keras.models import load_model
# Save the model
model_dbert.save('.//model_bert_base.h5')